In [ ]:
import glob
import datetime
import numpy as np

# for plotting
import matplotlib
import matplotlib.font_manager as fm
#!wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
fm.fontManager.ttflist += fm.createFontList(['thsarabunnew-webfont.ttf'])
matplotlib.rc('font', family='TH Sarabun New')
import matplotlib.pyplot as plt

# matplotlib date helper functions
from matplotlib.dates import (YEARLY, DateFormatter, datestr2num,
                              rrulewrapper, RRuleLocator, drange)

In [ ]:
tripod_files = glob.glob('./npy/Tripod*.npy')
tripod_files.sort()
print(len(tripod_files))
tripod_files

In [ ]:
# series 1 - use the mask from May 25 for all days
s1_start_idx = 3
data1 = np.load(tripod_files[s1_start_idx])
data2 = np.load(tripod_files[s1_start_idx+1])
data3 = np.load(tripod_files[s1_start_idx+2])

iou1 = np.concatenate((np.expand_dims(data1, 1),
                       np.expand_dims(data2, 1),
                       np.expand_dims(data3, 1)), 1)
# for s1_start_idx = 0
'''
iou1 = np.concatenate((np.expand_dims(data1[:, 0], 1),
                       np.expand_dims(data2[:, 0], 1),
                       np.expand_dims(data3[:, 0], 1)), 1)
'''                       

# Version 1

In [ ]:
# FIGURE version 1 - series 1 only
ms = 8
font = 20

formatter = DateFormatter('%m-%d')
date1 = datetime.date(2019, 5, 22)
date2 = datetime.date(2019, 7, 5)
delta = datetime.timedelta(days=1)

dates = drange(date1, date2, delta)


fig, ax = plt.subplots(figsize=(6, 4))

#COL = 1

#plt.plot_date(dates, data1[:, 1], marker='o', markerfacecolor='none', c='k', markersize=8, label='C=2')
plt.plot_date(dates[3], iou1.mean(axis=1)[3], c='r', markersize=ms)

#plt.plot_date(dates, data1[:, 0], marker='o', markerfacecolor='none', c='k', markersize=8)
plt.errorbar(dates, iou1.mean(axis=1), yerr=iou1.std(axis=1), elinewidth=1, capsize=2,
             marker='o', markerfacecolor='none', c='k', markersize=ms, linestyle=' ')
#plt.plot_date(dates[3], data1[3, 0], c='r', markersize=8)

ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_tick_params(rotation=30, labelsize=font)
# ax.set_ylabel(r'\textbf{IoU}', fontsize=font) # LaTeX
#ax.set_xlabel(r'\textbf{Date}', fontsize=font)
ax.set_ylabel('IoU (C=1)', fontsize=font)  # LaTeX
ax.set_xlabel('Date (MM-DD-2019)', fontsize=font)
ax.tick_params(labelsize=font-2)
ax.grid()

# plt.legend()
plt.tight_layout()
plt.show()

figure_name = tripod_files[0].split('/')[-1].split('.')[0]
print(figure_name)
#fig.savefig(figure_name + 'v1.png', format='png')
#fig.savefig(figure_name + '.eps', format='eps')

# Version 2

In [ ]:
tripod_files

In [ ]:
# series 2 - use the ten different masks
s2_start_idx = 6
data1_s2 = np.load(tripod_files[s2_start_idx])
data2_s2 = np.load(tripod_files[s2_start_idx+1])
data3_s2 = np.load(tripod_files[s2_start_idx+2])

iou1_s2 = np.concatenate((np.expand_dims(data1_s2, 1),
                          np.expand_dims(data2_s2, 1),
                          np.expand_dims(data3_s2, 1)), 1)

In [ ]:
# first, match the dates for labeled images to the DateFormatter axis
root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Tripod/1352/biofouling/'
jpeg_files = glob.glob(root_path + '*.jpg')
jpeg_files.sort()
dates_s2 = np.asarray([datestr2num(f.split('/')[-1].split('_')[2]) for f in jpeg_files])

In [ ]:
# FIGURE version 2 - series 1 and 2
ms = 8
font = 20
cs = 3

# Calculated using notebook "task_3_evaluate_checkpoint_on_tripod"
INTER_ANNOTATOR_AGREEMENT = 0.40643012523651123

formatter = DateFormatter('%m-%d')
date1 = datetime.date(2019, 5, 22)
date2 = datetime.date(2019, 7, 5)
delta = datetime.timedelta(days=1)

dates = drange(date1, date2, delta)


fig, ax = plt.subplots(figsize=(7, 5))

#plt.plot_date(dates[3], iou1.mean(axis=1)[3], c='r', markersize=ms, label='One day labeled')
# depict inter-annotator variability
#ax.hlines(INTER_ANNOTATOR_AGREEMENT, dates[0], dates[-1], linestyle='--', color='green', label='Human')
#ax.vlines(dates[3], 0, INTER_ANNOTATOR_AGREEMENT, linestyle='--', color='gray')
plt.plot_date(dates[3], INTER_ANNOTATOR_AGREEMENT, markerfacecolor='gold', c='y', markersize=3*ms, marker='*', label='May 25 inter-human')

# series 1
#plt.errorbar(dates[3], iou1.mean(axis=1)[3], yerr=iou1.std(axis=1)[3], elinewidth=1, capsize=cs,
#             marker='o', markerfacecolor='r', c='k', markersize=ms, linestyle=' ', label='May 25 label')

plt.errorbar(dates, iou1.mean(axis=1), yerr=iou1.std(axis=1), elinewidth=1, capsize=cs-1,
             marker='o', markerfacecolor='none', c='k', markersize=ms, linestyle=' ', label='IoU wrt May 25 label')

# series 2
plt.errorbar(dates_s2, iou1_s2.mean(axis=1), yerr=iou1_s2.std(axis=1), elinewidth=1, capsize=cs, capthick=1,
             marker='^', markerfacecolor='b', c='k', markersize=ms, linestyle=' ', label='IoU wrt same-day label')

#ax.set_xlim(dates[0], dates[-1])
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_tick_params(rotation=30, labelsize=font)
ax.set_ylabel('IoU (C=1)', fontsize=font)  # LaTeX
ax.set_xlabel('Date (MM-DD-2019)', fontsize=font)
ax.tick_params(labelsize=font-2)
ax.grid()

#plt.legend(fontsize=font-4)
#plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=4, fancybox=True, shadow=True, fontsize=font-8)
#plt.legend(loc='best', fontsize=font-7, fancybox=True, shadow=True)
plt.legend(bbox_to_anchor=(0.54, 1.05), fontsize=font-7, fancybox=True, shadow=True)

plt.tight_layout()
plt.show()

figure_name = tripod_files[4].split('/')[-1].split('.')[0]
print(figure_name)
fig.savefig(figure_name + '_v2-r2.png', format='png')
#fig.savefig(figure_name + '_v2-r1.eps', format='eps')

# Version 3 - In series 2, use the most recent labels for each day

In [ ]:
#tripod_files[7]

In [ ]:
# series 2 - use the ten different masks
base_idx = 7
data1_s2 = np.load(tripod_files[base_idx])
data2_s2 = np.load(tripod_files[base_idx + 1])
data3_s2 = np.load(tripod_files[base_idx + 2])

iou1_s2_full = np.concatenate((np.expand_dims(data1_s2, 1),
                               np.expand_dims(data2_s2, 1),
                               np.expand_dims(data3_s2, 1)), 1)

In [ ]:
mask = (dates[:, None] == dates_s2).any(axis=-1)

In [ ]:
# FIGURE version 3 - series 1 and 2
ms = 8
font = 20

formatter = DateFormatter('%m-%d')
date1 = datetime.date(2019, 5, 22)
date2 = datetime.date(2019, 7, 5)
delta = datetime.timedelta(days=1)

dates = drange(date1, date2, delta)


fig, ax = plt.subplots(figsize=(7, 5))

#plt.plot_date(dates[3], iou1.mean(axis=1)[3], c='r', markersize=ms, label='One day labeled')

# series 1
plt.errorbar(dates[3], iou1.mean(axis=1)[3], yerr=iou1.std(axis=1)[3], elinewidth=1, capsize=2,
             marker='o', markerfacecolor='r', c='k', markersize=ms, linestyle=' ', label='Labeled')
plt.errorbar(dates, iou1.mean(axis=1), yerr=iou1.std(axis=1), elinewidth=1, capsize=2,
             marker='o', markerfacecolor='none', c='k', markersize=ms, linestyle=' ', label='Unlabeled')

# series 2
plt.errorbar(dates[mask], iou1_s2_full.mean(axis=1)[mask], yerr=iou1_s2_full.std(axis=1)[mask], elinewidth=1, capsize=3, 
             marker='^', markerfacecolor='b', c='k', markersize=ms, linestyle=' ', label='Labeled')

plt.errorbar(dates, iou1_s2_full.mean(axis=1), yerr=iou1_s2_full.std(axis=1),elinewidth=1, capsize=3,
             marker='^', markerfacecolor='none', c='k', markersize=ms, linestyle=' ', label='Unlabeled')

ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_tick_params(rotation=30, labelsize=font)
ax.set_ylabel('IoU (C=1)', fontsize=font)  # LaTeX
ax.set_xlabel('Date (MM-DD-2019)', fontsize=font)
ax.tick_params(labelsize=font-2)
ax.grid()

# plt.legend(fontsize=font-4)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15),
           ncol=4, fancybox=True, shadow=True, fontsize=font-8)
plt.tight_layout()
plt.show()

figure_name = tripod_files[4].split('/')[-1].split('.')[0]
print(figure_name)
#fig.savefig(figure_name + 'v3.png', format='png')
#fig.savefig(figure_name + '.eps', format='eps')